In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [3]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [176]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from sklearn.neural_network import MLPRegressor

In [66]:
from sklearn.metrics import r2_score

In [5]:
#import keras

In [6]:
total_data = pd.read_csv('Combined_Roto_NBA_18-19.csv')

/Users/jordanlevy/Desktop/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
total_data.columns

Index(['Date', 'Name', 'Salaries', 'Team', 'Position', 'Opponent',
       'Unnamed: 6', 'Projected Minutes', 'DvP', 'DvPRank', 'O/U', 'Line',
       'Total', 'Movement', 'Ceiling', 'Floor', 'Projection', 'Pts/$/K', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'FD_PTS'],
      dtype='object')

In [83]:
filt_data = total_data[total_data['MIN'] > 15].reset_index()

In [84]:
len(filt_data)

18734

In [85]:
feat_data = filt_data[['Date', 'Position', 'Salaries', 'Line', 'Projected Minutes', 'Ceiling', 'Floor', 'Projection', 'FD_PTS']]

In [86]:
feat_data = feat_data[feat_data['Line'] != '\xa0'].reset_index(drop=True)#['Line'].str.strip(' ')

In [87]:
feat_data['Line'] = feat_data['Line'].astype('int')

In [88]:
mean_data = feat_data.groupby('Date').transform(np.mean)
std_data = feat_data.groupby('Date').transform(np.std)

In [132]:
feat_data.head()

,Date,Position,Salaries,Line,Projected Minutes,Ceiling,Floor,Projection,FD_PTS
0,2018-10-16,PF,6400,180,33.0,37.61,18.53,28.07,11.7
1,2018-10-16,C,10400,180,33.0,59.02,33.20,46.11,42.0
2,2018-10-16,SG,5900,-215,32.0,34.31,18.07,26.19,19.0
3,2018-10-16,SF,6500,180,33.0,39.30,18.92,29.11,22.2
4,2018-10-16,PG,8400,-215,33.0,46.72,28.64,37.68,19.3


In [90]:
feat_scale_data = (feat_data.iloc[:,2:] - mean_data) / std_data 

In [91]:
feat_scale_data['Position'] = feat_data['Position'].values

In [92]:
feat_scale_data.head()

,Salaries,Line,Projected Minutes,Ceiling,Floor,Projection,FD_PTS,Position
0,0.081633,0.326864,0.829381,0.130084,0.090917,0.115736,-1.049892,PF
1,1.868310,0.326864,0.829381,1.748028,1.497421,1.679891,0.974554,C
2,-0.141702,-0.073802,0.676183,-0.119296,0.046814,-0.047269,-0.562154,SG
3,0.126300,0.326864,0.829381,0.257796,0.128309,0.205909,-0.348351,SF
4,0.974971,-0.073802,0.829381,0.818523,1.060225,0.948969,-0.542110,PG


In [244]:
mlp = MLPRegressor(hidden_layer_sizes = (100,), early_stopping=True)

In [245]:
X = feat_data[['Salaries','Projection', 'Projected Minutes','Ceiling', 'Floor', 'Line']]
y = feat_data[['FD_PTS']]

In [229]:
#r2_score(y, X)

In [230]:
#X = feat_scale_data[['Salaries','Projection', 'Projected Minutes', 'Projected Minutes']]#,'Projected Minutes']]#'Salaries', , 'Projection','Ceiling', 'Floor']]

In [231]:
#y = feat_scale_data[['FD_PTS']]

In [232]:
#r2_score(y, X)

In [247]:
score_arr = []
#for i in range(5):
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =np.random)
mlp.fit(X_train, y_train)
score = mlp.score(X_test, y_test)
#score_arr.append(score)
#print(i+1)
#np.mean(score_arr)
score

0.46557942097510296

In [129]:
mlp.fit(X_train, y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [170]:
mlp.predict(X_test)

array([17.09775407, 24.62785554, 38.65199641, 23.3001031 , 16.89704682,
       14.95845278, 17.81478416, 33.52472074, 25.07511412, 21.74052768,
       18.5368792 , 19.2317519 , 21.73711129, 21.66722275, 17.62621315,
       30.87086443, 15.06628983, 21.86875429, 24.93734337, 18.50505843,
       19.91071422, 21.22880327, 35.62430569, 46.5667777 , 41.46766791,
       31.05035784, 45.60590764, 17.24424468, 15.43753213, 16.8776008 ,
       16.3833161 , 14.85238355, 37.78930001, 16.91826067, 29.11600447,
       32.67156848, 18.62255887, 17.38060535, 29.46780075, 16.7875612 ,
       33.37528028, 15.63552803, 22.04388777, 21.21454146, 13.50530434,
       23.79686066, 16.43281507, 17.13735116, 30.4208945 , 16.46734719,
       13.58485626, 23.95950014, 25.38460196, 22.80405058, 28.10917108,
       19.69504012, 15.84236303, 19.09032627, 16.67866137, 21.74771821,
       27.78825204, 15.43753213, 20.62220235, 17.87724366, 29.61995256,
       22.60876604, 17.45638314, 28.6200712 , 19.86651871, 48.07

In [124]:
mlp.score(X_test, y_test)

0.5352854664902953

In [88]:
ohe = OneHotEncoder(sparse=False)

In [60]:
X = ohe.fit_transform(total_data[['Position']])

In [61]:
X

array([[0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]])